In [141]:
import pandas as pd 
import numpy as np

df = pd.read_excel(r"C:\Users\wallace.souza\Downloads\LOGISTICA 2.0 - CONTROLE FINANCEIRO DE LOGÍSTICA.xlsx")
fn = pd.read_excel(r"C:\Users\wallace.souza\Downloads\Fornecedores.xlsx")


In [142]:
df = df.merge(fn, on='Código Fornecedor:', how='left')

In [143]:
df = df[df['Hora de início'].notnull()]

In [144]:
pd.set_option('display.max_columns', None)


In [145]:
df = df[['Hora de conclusão','Setor:\n',
       'Total de Pallets:','Total de Volumes:',
       'Filmagem de Quantos Palets ?','Total de Palets MISTURADOS?','Lona Truck ?','Lona Carreta ?',
       'Total $ Recebido em CASH?', 'Total Recebido por PIX ?',
       'Valor total dos Serviços ?', 'Valor Pago:', 'Valor da Diferença:',
      'Código Fornecedor:','Nome do Fornecedor:_x','Nome do Fornecedor:_y', 'Nome da Transportadora:',    
       'Data do Recebimento dos Valores:', 'Data do Serviço:',
       'Constava agendamento para hoje:',
        'Reg']]

df = df.rename(columns={'Nome do Fornecedor:_x':'Nome do Fornecedor: - Ivo'})
df = df.rename(columns={'Nome do Fornecedor:_y':'Nome do Fornecedor:'})

In [146]:
df['Valor Pago:'] = pd.to_numeric(df['Valor Pago:'], errors='coerce')


In [147]:
df['Valor total dos Serviços ?'] = pd.to_numeric(df['Valor total dos Serviços ?'], errors='coerce')


In [148]:
df[df['Valor Pago:'].isna()]
df['Nome do Fornecedor:'] = df['Nome do Fornecedor:'].str.strip()

In [149]:
df['%'] = df['Valor Pago:'] / df['Valor total dos Serviços ?'] * 100 - 100
df['Taxa'] = np.where(df['Total Recebido por PIX ?'] != 0, 'Taxa Pix?', "Ok")
df['Total de Pallets:'] = df['Total de Pallets:'].astype(float)
df['Total de Volumes:'] = df['Total de Volumes:'].astype(float)
df['Filmagem de Quantos Palets ?'] = df['Filmagem de Quantos Palets ?'].astype(float)

peso_pallet = np.where(df['Setor:\n'] == 'Perecíveis', 30, 25)
fornecedor = np.where(df['Nome do Fornecedor:'] == 'AMBEV S/A',20, peso_pallet)
lona = np.where(df['Lona Carreta ?'] == 'Sim',70, 0)
deslonamento = np.where(df['Lona Truck ?'] == 'Sim',50, 0)

df['Total de Volumes:']
	

denominador = (
        df['Total de Pallets:'] * fornecedor +
        df['Total de Volumes:'] + deslonamento +
        df['Filmagem de Quantos Palets ?'] * 20 +
        df['Total de Palets MISTURADOS?'] * 60 + lona
        
    )
    
# Cálculo da coluna final
df['Valor Pallet'] = df['Valor total dos Serviços ?'] / denominador
df['Valor Correto'] = denominador


In [150]:
colunas = ['Total $ Recebido em CASH?', 'Total Recebido por PIX ?',
           'Valor total dos Serviços ?', 'Valor Pago:', 'Valor da Diferença:']

df[colunas] = df[colunas].apply(
    lambda col: pd.to_numeric(col, errors='coerce')
)


In [151]:
df['Diferença'] = df['Valor total dos Serviços ?'] - df['Valor Correto']
df['Pix + Cash'] = (df['Total $ Recebido em CASH?'] + df['Total Recebido por PIX ?']) / df['Valor Pago:']


In [152]:
df['Data'] = df['Data do Recebimento dos Valores:'].dt.month

In [153]:
df1 = df[df['Data']== 11]

In [154]:
df1['Reg'] = df1['Reg'].astype(int) 
df1['Código Fornecedor:'] = df1['Código Fornecedor:'].astype(int) 

C:\Users\wallace.souza\AppData\Local\Temp\ipykernel_31656\2778285773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Reg'] = df1['Reg'].astype(int)
C:\Users\wallace.souza\AppData\Local\Temp\ipykernel_31656\2778285773.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Código Fornecedor:'] = df1['Código Fornecedor:'].astype(int)


In [155]:
df1[(df1['Valor Pallet'] != 1) | (df1['Pix + Cash'] != 1) | (df1["Nome do Fornecedor:"].isna())]

,Hora de conclusão,Setor:\n,Total de Pallets:,Total de Volumes:,Filmagem de Quantos Palets ?,Total de Palets MISTURADOS?,Lona Truck ?,Lona Carreta ?,Total $ Recebido em CASH?,Total Recebido por PIX ?,Valor total dos Serviços ?,Valor Pago:,Valor da Diferença:,Código Fornecedor:,Nome do Fornecedor: - Ivo,Nome do Fornecedor:,Nome da Transportadora:,Data do Recebimento dos Valores:,Data do Serviço:,Constava agendamento para hoje:,Reg,%,Taxa,Valor Pallet,Valor Correto,Diferença,Pix + Cash,Data
18714,2025-11-06 17:04:13,Mercearia,60.0,0.0,0.0,0.0,Não,Não,1800.0,0.0,1500.0,1800.0,0.0,1845,MFL,NaN,PROPRIO,2025-11-06,2025-11-06,Não,18716,20.0,Ok,1.0,1500.0,0.0,1.0,11


In [156]:
noshow = df[(df['Constava agendamento para hoje:']=='Sim, No Show pendente') | (df['%']>=30)]
nao = df[(df['Constava agendamento para hoje:'].str.contains('não', na=False, case=False))]

In [157]:
print('pagamento em pix:' ,len(df[df['Total Recebido por PIX ?'] !=0]))
print('pagamento em dinheiro:' , len(df[df['Total Recebido por PIX ?'] ==0]))

pagamento em pix: 1493
pagamento em dinheiro: 1395


In [158]:
print('noshow = pagamento em pix:', len(noshow[noshow['Total Recebido por PIX ?'] !=0]))
print('noshow = pagamento em dinheiro:' , len(noshow[noshow['Total Recebido por PIX ?'] ==0]))

noshow = pagamento em pix: 237
noshow = pagamento em dinheiro: 47


In [159]:
print('não agendado = pagamento em pix:', len(nao[nao['Total Recebido por PIX ?'] !=0]))
print('não agendado = pagamento em dinheiro:' , len(nao[nao['Total Recebido por PIX ?'] ==0]))

não agendado = pagamento em pix: 124
não agendado = pagamento em dinheiro: 55


In [160]:
df1[df1['Data do Serviço:'] != df1['Data do Recebimento dos Valores:']]

,Hora de conclusão,Setor:\n,Total de Pallets:,Total de Volumes:,Filmagem de Quantos Palets ?,Total de Palets MISTURADOS?,Lona Truck ?,Lona Carreta ?,Total $ Recebido em CASH?,Total Recebido por PIX ?,Valor total dos Serviços ?,Valor Pago:,Valor da Diferença:,Código Fornecedor:,Nome do Fornecedor: - Ivo,Nome do Fornecedor:,Nome da Transportadora:,Data do Recebimento dos Valores:,Data do Serviço:,Constava agendamento para hoje:,Reg,%,Taxa,Valor Pallet,Valor Correto,Diferença,Pix + Cash,Data
18622,2025-11-03 11:13:51,Mercearia,41.0,0.0,0.0,0.0,Não,Não,0.0,1127.5,1025.0,1127.5,0.0,1323,ROMANATO,ROMANATO ALIMENTOS LTDA,PROPRIO,2025-11-03,2025-10-30,Sim,18624,10.0,Taxa Pix?,1.0,1025.0,0.0,1.0,11
18675,2025-11-05 12:14:30,Mercearia,14.0,0.0,0.0,0.0,Não,Não,0.0,423.5,350.0,423.5,0.0,1093,UNILIDER,UNILIDER DISTRIBUIDORA S.A.,PROPRIO,2025-11-05,2025-11-04,Sim,18677,21.0,Taxa Pix?,1.0,350.0,0.0,1.0,11
18837,2025-11-13 09:35:43,Perecíveis,32.0,0.0,4.0,0.0,Não,Não,1144.0,0.0,1040.0,1144.0,0.0,2676848,NEW BRÁZ,NEW BRAZ IMPORT & EXPORT LTDA,PROPRIO,2025-11-13,2025-11-04,Sim,18839,10.0,Ok,1.0,1040.0,0.0,1.0,11


In [161]:
b = df1[(df1['Taxa'] == 'Taxa Pix?') & (df1['%'] == 0)]

In [162]:
b["Nome do Fornecedor:"].value_counts().reset_index()

,Nome do Fornecedor:,count
0,AMBEV S/A,15
1,BRF S.A.,10


In [163]:
df1[(df1["Data do Recebimento dos Valores:"].dt.day == 18)]

,Hora de conclusão,Setor:\n,Total de Pallets:,Total de Volumes:,Filmagem de Quantos Palets ?,Total de Palets MISTURADOS?,Lona Truck ?,Lona Carreta ?,Total $ Recebido em CASH?,Total Recebido por PIX ?,Valor total dos Serviços ?,Valor Pago:,Valor da Diferença:,Código Fornecedor:,Nome do Fornecedor: - Ivo,Nome do Fornecedor:,Nome da Transportadora:,Data do Recebimento dos Valores:,Data do Serviço:,Constava agendamento para hoje:,Reg,%,Taxa,Valor Pallet,Valor Correto,Diferença,Pix + Cash,Data
18933,2025-11-18 09:28:24,Mercearia,12.0,0.0,0.0,0.0,Não,Não,300.0,0.0,300.0,300.0,0.0,11675,RECREIO,RECREIO RIO DISTRIBUIDORA DE ALIMENTOS LTDA,PROPRIO,2025-11-18,2025-11-18,Sim,18935,0.0,Ok,1.0,300.0,0.0,1.0,11
18934,2025-11-18 10:44:53,Mercearia,26.0,0.0,0.0,0.0,Não,Não,0.0,520.0,520.0,520.0,0.0,1003,AMBEV,AMBEV S/A,FADEL,2025-11-18,2025-11-18,Sim,18936,0.0,Taxa Pix?,1.0,520.0,0.0,1.0,11
18935,2025-11-18 10:46:48,Mercearia,26.0,0.0,0.0,0.0,Não,Não,650.0,0.0,650.0,650.0,0.0,1012,COPAPA,COPAPA CIA DE PAPEIS,PROPRIO,2025-11-18,2025-11-18,Sim,18937,0.0,Ok,1.0,650.0,0.0,1.0,11
18936,2025-11-18 10:47:38,Mercearia,26.0,0.0,0.0,0.0,Não,Não,650.0,0.0,650.0,650.0,0.0,1012,COPAPA,COPAPA CIA DE PAPEIS,PROPRIO,2025-11-18,2025-11-18,Sim,18938,0.0,Ok,1.0,650.0,0.0,1.0,11
18937,2025-11-18 12:23:14,Mercearia,6.0,0.0,0.0,0.0,Não,Não,150.0,0.0,150.0,150.0,0.0,1121,HNK,HNK BR INDUSTRIA DE BEBIDAS LTDA,PROPRIO,2025-11-18,2025-11-18,Sim,18939,0.0,Ok,1.0,150.0,0.0,1.0,11
18938,2025-11-18 12:49:44,Mercearia,19.0,0.0,0.0,0.0,Não,Não,0.0,522.5,475.0,522.5,0.0,1112,EBD,EMPRESA BRASILEIRA DE DISTRIBUICAO LTDA,PROPRIO,2025-11-18,2025-11-18,Sim,18940,10.0,Taxa Pix?,1.0,475.0,0.0,1.0,11
18939,2025-11-18 12:50:38,Mercearia,29.0,0.0,0.0,0.0,Não,Não,725.0,0.0,725.0,725.0,0.0,1496,M DIAS,M DIAS BRANCO S.A. IND. E COM.,PROPRIO,2025-11-18,2025-11-18,Sim,18941,0.0,Ok,1.0,725.0,0.0,1.0,11
18940,2025-11-18 13:00:20,Mercearia,3.0,0.0,0.0,0.0,Não,Não,75.0,0.0,75.0,75.0,0.0,1299,SUPER GIRO,SUPER GIRO DIST.DE GEN.ALIMENTICIOS LTDA,PROPRIO,2025-11-18,2025-11-18,Sim,18942,0.0,Ok,1.0,75.0,0.0,1.0,11
18941,2025-11-18 13:31:52,Mercearia,7.0,0.0,0.0,0.0,Não,Não,0.0,192.5,175.0,192.5,0.0,1118,CCN,CCN COM. CENTRO NORTE ALIMENTOS LTDA,PROPRIO,2025-11-18,2025-11-18,Sim,18943,10.0,Taxa Pix?,1.0,175.0,0.0,1.0,11
18942,2025-11-18 14:58:31,Perecíveis,5.0,0.0,0.0,23.0,Não,Não,0.0,1530.0,1530.0,1530.0,0.0,1006,BRF,BRF S.A.,PROPRIO,2025-11-18,2025-11-18,Sim,18944,0.0,Taxa Pix?,1.0,1530.0,0.0,1.0,11
